In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io #Used as buffer

In [2]:
nn_p=5
nn_w = 5
nn_tw = 1

# Functions

In [3]:
path= Path(f'../../data/cluster_results')
col_by_freq = True
col_palette = 'Set3'
pat = re.compile('.*/(exp\d)/(\w*)/nonn/freq([0-9\.]+)div(\w+)/(\w+)_\w+.csv')
decode_keys = ['experiment','function','freq','div','method']
# nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
# nn_decode_keys = ['experiment','function','freq','div','method','replace_mech']

nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)nn_w(\d+)nn_p(\d+)\w+nn_tw(\d+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
nn_decode_keys = ['experiment','function','freq','nnw','nnp','nntw','div','method','replace_mech']#,


def get_nn_files(m): return list(path.glob(f'**/nn/**/*{m}.csv'))


def read_nn_csv(f,m):
    df = pd.read_csv(f)
    for k,v in zip(nn_decode_keys,nn_pat.search(str(f)).groups()): df[k] = v
    df['freq'] = df['freq'].astype(float)
    df['method'] = df['method'].str.replace('NNnorm_Worst', 'NN')
    df['method'] = df['method'] + '_' + df['div'] #'_' + df['replace_mech'] +
    
    df.drop(['replace_mech','div'], axis=1, inplace=True)
    df.rename({'0':m.upper(), m:m.upper()}, axis=1, inplace=True)
    return df

def get_data(m, normalize=False):
    nn_files = get_nn_files(m)
    data = pd.concat([read_nn_csv(f,m) for f in nn_files])
    data = data[data['nnw']==str(nn_w)]
    data = data[data['nntw']==str(nn_tw)]
    data = data[data['nnp']==str(nn_p)]
    if normalize:
        data_norm = (data.groupby(['experiment','function','freq','method'])[m.upper()].mean().reset_index()
                         .groupby(['experiment','function'])[m.upper()].min().reset_index()
                         .rename({m.upper():m.upper()+'_norm'}, axis=1))
        data = data.merge(data_norm, 'left')
        data[m.upper()+'_norm'] = data[m.upper()] / data[m.upper()+'_norm']
    return data




# Table

In [4]:
df=get_data('nn_time', normalize=False)
df['function']=df['function'].str.title()
df.head()

,NN_TIME,experiment,function,freq,nnw,nnp,nntw,method
0,0.005685,exp1,Sphere,20.0,5,5,1,NN_HMu
1,0.005639,exp1,Sphere,20.0,5,5,1,NN_HMu
2,0.005623,exp1,Sphere,20.0,5,5,1,NN_HMu
3,0.005611,exp1,Sphere,20.0,5,5,1,NN_HMu
4,0.005553,exp1,Sphere,20.0,5,5,1,NN_HMu


In [5]:
table2=df.pivot_table(values='NN_TIME', index=['function','method'] ,columns=['experiment','freq'])
table2

experiment             exp1                                                    \
freq                   1.0       5.0       10.0      20.0      30.0      40.0   
function   method                                                               
Rastrigin  NN_CwN  0.107633  0.021536  0.011239  0.005540       NaN       NaN   
           NN_HMu  0.111064  0.022273  0.011166  0.005795  0.003732  0.002820   
           NN_No   0.107551  0.021357  0.010851  0.005480  0.003599  0.002729   
           NN_RI   0.110703  0.021625  0.011172  0.005546  0.003704  0.002779   
           NN_Rst  0.108759  0.021417  0.010649  0.005494  0.004926  0.002704   
Rosenbrock NN_CwN  0.106584  0.021217  0.010788  0.005388       NaN       NaN   
           NN_HMu  0.108259  0.021966  0.011061  0.005582  0.003723  0.002827   
           NN_No   0.108618  0.022060  0.010909  0.005447  0.003681  0.002732   
           NN_RI   0.109208  0.021725  0.010873  0.005613  0.003785  0.002729   
           NN_Rst  0.108161  0.021347  0.010914  0.005483  0.003618  0.002675   
Sphere     NN_CwN  0.107570  0.021358  0.010670  0.005346       NaN       NaN   
           NN_HMu  0.108140  0.022203  0.010995  0.005587  0.003770  0.002809   
           NN_No   0.109524  0.022107  0.011279  0.005604  0.003765  0.002773   
           NN_RI   0.109811  0.021792  0.011111  0.005621  0.003756  0.002792   
           NN_Rst  0.107171  0.021253  0.010697  0.005538  0.003598  0.002691   

experiment                       exp2                      ...      exp3  \
freq                   50.0      1.0       5.0       10.0  ...      30.0   
function   method                                          ...             
Rastrigin  NN_CwN       NaN  0.109409  0.021441  0.011289  ...       NaN   
           NN_HMu  0.002224  0.109280  0.022244  0.011497  ...  0.003747   
           NN_No   0.002488  0.109514  0.021899  0.010970  ...  0.003523   
           NN_RI   0.002897  0.108962  0.021886  0.011171  ...  0.003720   
           NN_Rst  0.002175  0.109490  0.021389  0.010656  ...  0.003660   
Rosenbrock NN_CwN       NaN  0.107724  0.021673  0.011403  ...       NaN   
           NN_HMu  0.002215  0.108781  0.022163  0.011625  ...  0.003778   
           NN_No   0.002305  0.108232  0.021732  0.011006  ...  0.003655   
           NN_RI   0.002854  0.109822  0.021860  0.011035  ...  0.003621   
           NN_Rst  0.002205  0.108080  0.021642  0.010892  ...  0.003727   
Sphere     NN_CwN       NaN  0.108272  0.021643  0.011712  ...       NaN   
           NN_HMu  0.002194  0.107515  0.021827  0.011387  ...  0.003738   
           NN_No   0.002357  0.108298  0.029883  0.010977  ...  0.003608   
           NN_RI   0.002898  0.112290  0.021861  0.015039  ...  0.003675   
           NN_Rst  0.002189  0.107453  0.021592  0.010582  ...  0.003666   

experiment                                 exp4                                \
freq                   40.0      50.0      1.0       5.0       10.0      20.0   
function   method                                                               
Rastrigin  NN_CwN       NaN       NaN  0.106107  0.021026  0.010794  0.007278   
           NN_HMu  0.002737  0.002849  0.106080  0.021437  0.010742  0.005402   
           NN_No   0.002703  0.002228  0.106281  0.021391  0.010563  0.005533   
           NN_RI   0.002787  0.002257  0.105173  0.021238  0.010758  0.005486   
           NN_Rst  0.003183  0.002183  0.106091  0.021165  0.010766  0.005376   
Rosenbrock NN_CwN       NaN       NaN  0.108032  0.021128  0.010820  0.005294   
           NN_HMu  0.002779  0.002239  0.106741  0.021520  0.011068  0.005414   
           NN_No   0.002789  0.002254  0.105535  0.021634  0.011081  0.005467   
           NN_RI   0.002789  0.002205  0.105980  0.022333  0.011016  0.005452   
           NN_Rst  0.003225  0.002198  0.105880  0.021430  0.010668  0.005511   
Sphere     NN_CwN       NaN       NaN  0.105188  0.021002  0.011391  0.005303   
           NN_HMu  0.002788  0.002186 

In [18]:
print(table2.to_csv(float_format='%.3f'))

experiment,,exp1,exp2,exp3,exp4
freq,,10.0,10.0,10.0,10.0
function,method,,,,
Rastrigin,NN_HMu,0.011,0.011,0.011,0.011
Rastrigin,NN_No,0.011,0.011,0.011,0.011
Rastrigin,NN_RI,0.011,0.011,0.011,0.011
Rastrigin,NN_Rst,0.011,0.011,0.011,0.011
Rosenbrock,NN_HMu,0.011,0.011,0.011,0.011
Rosenbrock,NN_No,0.011,0.011,0.011,0.011
Rosenbrock,NN_RI,0.011,0.011,0.011,0.011
Rosenbrock,NN_Rst,0.011,0.011,0.011,0.011
Sphere,NN_HMu,0.011,0.011,0.011,0.011
Sphere,NN_No,0.011,0.011,0.011,0.011
Sphere,NN_RI,0.011,0.011,0.011,0.011
Sphere,NN_Rst,0.011,0.011,0.011,0.011

